# Linear Algebra - solving least squares problems
### Douglas Bates

# Linear algebra and statistical models

`Julia` provides one of the best, if not _the best_, environments for numerical linear algebra.

The `Base` package provides basic array (vector, matrix, etc.) construction and manipulation; `*`, `/`, `\`, `'`.  The `LinearAlgebra` package provides many definitions of matrix types (`Diagonal`, `UpperTriangular`, ...) and factorizations.  The `SparseArrays` packages provides types and methods for sparse matrices.

In [ ]:
A = rand(4,3)   # simulate a matrix with elements selected at random from (0,1)

In [ ]:
B = rand(3, 4)

In [ ]:
A'   # "lazy" transpose

In [ ]:
A*B

## Solving least squares problems

A least squares solution is one of the building blocks for statistical models.

If `X` is an $n\times p$ model matrix and `y` is an $n$-dimensional vector of observed responses, a _linear model_ is of the form
$$
\mathcal{Y}\sim\mathcal{N}(\mathbf{X}\beta, \sigma^2\mathbf{I})
$$

That is, the _mean response_ is modeled as a _linear predictor_ , $\mathbf{X}\beta$, depending on the _parameter vector_, $\beta$ (also called _coefficients_) with the covariance matrix, $\sigma^2\mathbf{I}$.  In general the probability density for a [multivariate normal distribution](https://en.wikipedia.org/wiki/Multivariate_normal_distribution) with mean $\mu$ and covariance matrix $\Sigma$ is
$$
  f(\mathbf{y}|\mu,\Sigma) = \frac{1}{(2\pi)^{n/2}|\Sigma|^{1/2}}
     \exp\left(-\frac{(\mathbf{y}-\mu)'\Sigma^{-1}(\mathbf{y}-\mu)}{2}\right)
$$
where $|\Sigma|$ denotes the determinant of $\Sigma$.

When the covariance matrix is of the form $\sigma^2\mathbf{I}$ the distribution is called a _spherical_ normal
distribution because the contours of constant density are $n$-dimensional spheres centered at $\mu$.
In these cases the probability density can be simplified to
$$
\begin{aligned}
   f(\mathbf{y}|\beta,\sigma)&= \frac{1}{(2\pi\sigma^2)^{n/2}} \exp\left(-\frac{(\mathbf{y}-\mathbf{X}\beta)'(\mathbf{y}-\mathbf{X}\beta)}{2\sigma^2}\right)\newline
   &= \frac{1}{(2\pi\sigma^2)^{n/2}} \exp\left(-\frac{\|\mathbf{y}-\mathbf{X}\beta)\|^2}{2\sigma^2}\right) .
\end{aligned}
$$

The _likelihood_ of the parameters, $\beta$ and $\sigma$, given the data, $\mathbf{y}$ (and, implicitly, $\mathbf{X}$), is the same expression as the density but with the roles of the parameters and the observations reversed
$$
L(\beta,\sigma|\mathbf{y})=\frac{1}{(2\pi\sigma^2)^{n/2}} \exp\left(-\frac{\|\mathbf{y}-\mathbf{X}\beta)\|^2}{2\sigma^2}\right) .
$$

The _maximum likelihood estimates_ of the parameters are the values that maximize the likelihood given the data.  It is convenient to maximize the logarithm of the likelihood, called the _log-likelihood_, instead of the likelihood.
$$
\ell(\beta,\sigma|\mathbf{y})=\log L(\beta,\sigma|\mathbf{y})=
-\frac{n}{2}\log(2\pi\sigma^2)-\frac{\|\mathbf{y}-\mathbf{X}\beta)\|^2}{2\sigma^2}
$$

(Because the logarithm function is monotone increasing, the values of $\beta$ and $\sigma$ that maximize the log-likelihood also maximize the likelihood.)

For any value of $\sigma$ the value of $\beta$ that maximizes the log-likelihood is the value that minimizes the sum of squared residuals,
$$
\widehat{\beta}=\arg\min_\beta \|\mathbf{y} - \mathbf{X}\beta\|^2
$$

## A simple linear regression model

Data from a calibration experiment on the optical density versus the concentration of Formaldehyde are available as the `Formaldehyde` data in `R`.  We use the `RCall` package to retrieve these data from `R`.

In [ ]:
using LinearAlgebra, RCall, StatsModels, Tables

In [ ]:
Formaldehyde = rcopy(R"Formaldehyde")

In [ ]:
R"""
library(ggplot2)
qplot(x=carb, y=optden, data=Formaldehyde, geom="point")
"""

In a _simple linear regression_ the model matrix, $\mathbf{X}$, consists of a column of 1's and a column of the covariate values; `carb`, in this case.

In [ ]:
X = hcat(ones(size(Formaldehyde, 1)), Formaldehyde.carb)

In [ ]:
y = Formaldehyde.optden

In [ ]:
β = X\y     # least squares estimate

In [ ]:
r = y - X*β   #residual

One of the conditions for $\hat{\beta}$ being the least squares estimate is that the residuals must be orthogonal to the columns of $\mathbf{X}$

In [ ]:
X'r   # not exactly zero but very small entries

## Creating the model matrix from a formula

Creating model matrices from a data table can be a tedious and error-prone operation.  In addition, _statistical inference_ regarding a linear model often considers groups of columns generated by model _terms_.  The `GLM` package provides methods to fit and analyze linear models and generalized linear models (described later) using a _formula/data_ specification similar to that in _R_.

In [ ]:
using GLM

In [ ]:
m1 = fit(LinearModel, @formula(optden ~ 1 + carb), Formaldehyde)

The evaluation of the formula is performed by the `StatsModels` package in stages.

In [ ]:
f1 = @formula(optden ~ 1 + carb)
y, X = modelcols(apply_schema(f1, schema(Formaldehyde)), Formaldehyde)

In [ ]:
X

## Matrix decompositions for least squares

According to the formulas given in text books, the least squares estimates are calculated as
$$
\widehat{\mathbf{\beta}}=\mathbf{X^\prime X}^{-1}\mathbf{X^\prime y}
$$

In practice, this formula is not the way the estimates are calculated, because it is wasteful to evaluate the inverse of a matrix if you just want to solve a system of equations.

Recall that the least squares estimate satisfies the condition that the residual is orthogonal to the columns of $\mathbf{X}$.
$$
\mathbf{X^\prime (y - X\widehat{\beta})} = \mathbf{0}
$$
which can be re-written as
$$
\mathbf{X^\prime X}\widehat{\mathbf{\beta}}=\mathbf{X^\prime y}
$$
These are called the _normal equations_ - "normal" in the sense of orthogonal, not in the sense of the normal distribution.

The matrix $\mathbf{X^\prime X}$ is symmetric and _positive definite_.  The latter condition means that
$$
\mathbf{v^\prime(X^\prime X)v}=\mathbf{(Xv)^\prime Xv} = \|\mathbf{Xv}\|^2 > 0\quad\forall \mathbf{v}\ne\mathbf{0}
$$
if $\mathbf{X}$ has full column rank.

We will assume that the model matrices $\mathbf{X}$ we will use do have full rank.  It is possible to handle rank-deficient model matrices but we will not cover that here.

A positive-definite matrix has a "square root" in the sense that there is a $p\times p$ matrix $\mathbf{A}$ such that
$$\mathbf{A^\prime A}=\mathbf{X^\prime X} .$$
In fact, when $p>1$ there are several.  A specific choice of $\mathbf{A}$ is an upper triangular matrix with positive elements on the diagonal, usually written $\mathbf{R}$ and called the upper Cholesky factor of $\mathbf{X^\prime X}$.

In [ ]:
X

In [ ]:
xpx = X'X

In [ ]:
ch = cholesky(xpx)

In [ ]:
ch.U'ch.U

In [ ]:
ch.U'ch.U ≈ xpx

Because the Cholesky factor is triangular, it is possible to solve systems of equations of the form
$$\mathbf{R^\prime R}\widehat{\mathbf{\beta}}=\mathbf{X^\prime y}$$
in place in two stages.  First solve for $\mathbf{v}$ in
$$\mathbf{R^\prime v}=\mathbf{X^\prime y}$$

In [ ]:
v = ldiv!(ch.U', X'y)

then solve for $\widehat{\mathbf{\beta}}$ in
$$
\mathbf{R}\widehat{\mathbf{\beta}}=\mathbf{v}
$$

In [ ]:
βc = ldiv!(ch.U, copy(v))     # solution from the Cholesky factorization

In [ ]:
βc ≈ β

These steps are combined in one of the many `LinearAlgebra` methods for solutions of equations.

In [ ]:
ldiv!(ch, X'y)

## Sum of squared residuals as a quadratic form

Another way of approaching the least squares problem is to write the sum of squared residuals as what is called a _quadratic form_.
$$
\begin{aligned}
r^2(\mathbf{\beta}) & = \|\mathbf{y} - \mathbf{X\beta}\|^2\\
&=\left\|\begin{bmatrix}\mathbf{X}&\mathbf{y}\end{bmatrix}\begin{bmatrix}\mathbf{-\beta}\\ 1\end{bmatrix}\right\|^2\\
&=\begin{bmatrix}\mathbf{-\beta}&1\end{bmatrix}\begin{bmatrix}\mathbf{X^\prime X} & \mathbf{X^\prime y}\\
  \mathbf{y^\prime X}&\mathbf{y^\prime y}\end{bmatrix}
  \begin{bmatrix}\mathbf{-\beta}\\ 1\end{bmatrix}\\
&=\begin{bmatrix}\mathbf{-\beta}&1\end{bmatrix}
  \begin{bmatrix}
    \mathbf{R_{XX}}^\prime&\mathbf{0}\\
    \mathbf{r_{Xy}}^\prime&r_{\mathbf{yy}}
  \end{bmatrix}
  \begin{bmatrix}
    \mathbf{R_{XX}}&\mathbf{r_{Xy}}\\
    \mathbf{0}&r_{\mathbf{yy}}
  \end{bmatrix}
  \begin{bmatrix}\mathbf{-\beta}\\ 1\end{bmatrix}\\
&= \left\|  \begin{bmatrix}
    \mathbf{R_{XX}}&\mathbf{r_{Xy}}\\
    \mathbf{0}&r_{\mathbf{yy}}
  \end{bmatrix}
  \begin{bmatrix}\mathbf{-\beta}\\ 1\end{bmatrix}\right\|^2\\
&= \|\mathbf{r_{Xy}}-\mathbf{R_{XX}\beta}\|^2 + r_{\mathbf{yy}}^2
\end{aligned}
$$
where $\begin{bmatrix}\mathbf{R_{XX}}&\mathbf{r_{Xy}}\\ \mathbf{0}&r_{\mathbf{yy}}\end{bmatrix}$ is the upper Cholesky factor of the augmented matrix
$\begin{bmatrix}\mathbf{X^\prime X} & \mathbf{X^\prime y}\\ \mathbf{y^\prime X}&\mathbf{y^\prime y}\end{bmatrix}$.

In [ ]:
Xy = hcat(X, y)              # augmented model matrix

In [ ]:
cha = cholesky(Xy'Xy)        # augmented Cholesky factor

Note that $\mathbf{R_{XX}}$ is just the Cholesky factor of $\mathbf{X^\prime X}$ which was previously calculated and the vector $\mathbf{r_{Xy}}$ is the solution to $\mathbf{R^\prime v}=\mathbf{X^\prime y}$.  The minimum sum of squares is $r^{\mathbf{yy}}$ which is attained when $\mathbf{\beta}$ is the solution to
$$\mathbf{R_{XX}}\widehat{\beta}=\mathbf{r_{Xy}}$$

In [ ]:
RXX = UpperTriangular(view(cha.U, 1:2, 1:2))

In [ ]:
RXX ≈ ch.U

In [ ]:
rXy = cha.U[1:2, end]     # creates a copy ("view" doesn't copy)

In [ ]:
rXy ≈ v

In [ ]:
βac = ldiv!(RXX, copy(rXy))     # least squares solution from the augmented Cholesky

In [ ]:
βac ≈ β

In [ ]:
abs2(cha.U[end,end]) ≈ sum(abs2, y - X*β)   # check on residual sum of squares

One reason for writing the least squares solution in this way is that we will use a similar decomposition for linear mixed models later.  Another is that when dealing with very large data sets we may wish to parallelize the calculation over many cores or over many processors.  The natural way to parallelize the calculation is in blocks of rows and the augmented Cholesky can be formed row by row using a `lowrankupdate`.

### A row-wise approach to least squares

To show this we create a row-oriented table from our `Formaldehyde` data set, which is in a column-oriented format.

In [ ]:
Formrows = Tables.rowtable(Formaldehyde)

then initialize a Cholesky factor and zero out its contents

In [ ]:
chr = cholesky(zeros(3, 3) + I)  # initialize

In [ ]:
fill!(chr.factors, 0);   # zero out the contents
chr

Update by rows

In [ ]:
fill!(chr.factors, 0)
for r in Formrows
    lowrankupdate!(chr, [1.0, r.carb, r.optden])
end
chr

This is the same augmented Cholesky factor as before but obtained in a different way.  For generalized linear mixed models and for nonlinear mixed models it can be an advantage to work row-wise when performing some of the least squares calculations.

### Other decompositions for least squares solutions

There are other ways of solving a least squares problem, such as using an _orthogonal-triangular_ decomposition, also called a _QR_ decomposition, or a singular value decomposition.  The bottom line is that we decompose $\mathbf{X}$ or $\mathbf{X^\prime X}$ into some convenient product of orthogonal or triangular or diagonal matrices and work with those.  Just to relate these ideas, the _QR_ decomposition of $\mathbf{X}$ is

In [ ]:
qrX = qr(X)

Notice that the `R` factor is the upper Cholesky factor of $\mathbf{X^\prime X}$ with the first row multiplied by -1 so its transposed product with itself is $\mathbf{X^\prime X}$.

In [ ]:
qrX.R'qrX.R

That is,

In [ ]:
qrX.R'qrX.R ≈ xpx

The original solution for $\widehat{\mathbf{\beta}}$ from the expression `X\y` is actually performed by taking a QR decomposition of `X`.